In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data.data_pipeline import DataPipeline

In [8]:
# Initialize pipeline
pipeline = DataPipeline(
    start_date='2010-01-01',
    end_date='2023-12-31',
    universe_size=500,
    cache_dir='data/cache',
    price_col='Close'
)

# Fetch data
symbols = ['AAPL', 'GOOGL', 'MSFT']
pipeline.fetch_data(symbols)

# Check status after fetch
print("\nStatus after fetch:")
print(pipeline.check_data_status())

# Process data
pipeline.process_data()

# Check status after processing
print("\nStatus after processing:")
print(pipeline.validate_data_quality())

# Create universe
pipeline.create_universe()

X_train, X_test, y_train, y_test = pipeline.get_training_data()

print("\nX_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

14:40:09 - DataPipeline - INFO - Fetching data for 3 symbols...
14:40:09 - DataPipeline - INFO - Successfully fetched data for 3 symbols

Status after fetch:
{'raw_data': {'count': 3, 'symbols': ['AAPL', 'MSFT', 'GOOGL'], 'sample_columns': ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']}, 'processed_data': {'count': 0, 'symbols': [], 'sample_columns': None}, 'universe': {'count': 0, 'symbols': []}}
14:40:09 - DataPipeline - INFO - Starting data processing with 3 symbols
14:40:09 - DataPipeline - INFO - Processed data for 3 symbols

Status after processing:
{'AAPL': {'missing_values': np.int64(60), 'data_points': 3522, 'start_date': Timestamp('2010-01-04 00:00:00-0500', tz='America/New_York'), 'end_date': Timestamp('2023-12-29 00:00:00-0500', tz='America/New_York'), 'avg_volume': np.float64(242283396.08177173), 'zero_volume_days': np.int64(0)}, 'MSFT': {'missing_values': np.int64(60), 'data_points': 3522, 'start_date': Timestamp('2010-01-04 00:00:00-0500', tz='Am